In [63]:
import math

#https://edwilliams.org/gccalc.htm

sk = 48.150245, 17.066950
sk2 = 48.340323, 17.696664

pt = 38.690840, -8.177875

#wgs84
#hdg 1-2 108.02042013
#hdg 2-1 305.58084939
#dist 2284.02804834

#FAI
#hdg 1-2 108.10249438
#hdg 2-1 305.66226711
#dist 2279.59044552

#drop old
#2296.3710722089704

from geopy.distance import great_circle
from geopy.distance import vincenty


GPS_MULT = 10000000.0
M_PI = 3.14159265358979323846

def distance_old(lat1, lon1, lat2, lon2):

    lat1 *= GPS_MULT
    lon1 *= GPS_MULT
    lat2 *= GPS_MULT
    lon2 *= GPS_MULT
    
    # Compute the average lat of lat1 and lat2 to get the width of a
    # 1 degree cell at that position of the earth:
    lat = (lat1 + lat2) / 2 / GPS_MULT * (M_PI / 180.0);

    # 111.3 km (in cm) is the width of 1 degree
    # disp.get_cos() is not usable here, because we deal with fractions of a degree.
    dx = math.cos(lat) * 111.3 * abs(lon1 - lon2) / GPS_MULT;
    dy = 1.0      * 111.3 * abs(lat1 - lat2) / GPS_MULT;

    return math.sqrt(dx * dx + dy * dy);

class CheapRuler(object):
    
    def init(self, latitude):
       
        cos = math.cos(latitude * M_PI / 180.);
        cos2 = 2. * cos * cos - 1.;
        cos3 = 2. * cos * cos2 - cos;
        cos4 = 2. * cos * cos3 - cos2;
        cos5 = 2. * cos * cos4 - cos3;

        # multipliers for converting longitude and latitude
        # degrees into distance (http://1.usa.gov/1Wb1bv7)
        self.kx = (111.41513 * cos - 0.09455 * cos3 + 0.00012 * cos5);
        self.ky = (111.13209 - 0.56605 * cos2 + 0.0012 * cos4);
    
    def dist(self, lat1, lon1, lat2, lon2):
        dx = (lon1 - lon2) * self.kx;
        dy = (lat1 - lat2) * self.ky;

        return math.sqrt(dx * dx + dy * dy) ;     
    
    def bearing(self, lat1, lon1, lat2, lon2):
        dx = (lon1 - lon2) * self.kx;
        dy = (lat1 - lat2) * self.ky;

        if (dx  == 0 and dy == 0):
            return 0

        value = math.atan2(dx, dy) * 180. / M_PI;

        if (value > 180.):
            value -= 360.
        

        return value;
        

lat1, lon1 = sk
lat2, lon2 = sk2

pt = sk2

vin = vincenty(sk, pt).kilometers
old = distance_old(lat1, lon1, lat2, lon2)
fai = great_circle(sk, pt).kilometers

cr = CheapRuler()
cr.init((lat1 + lat2) / 2)
cr = cr.dist(lat1, lon1, lat2, lon2)

print("vincenty\t", vin)
print("old\t\t", old, abs(old - vin))
print("great_circle\t", fai, abs(fai - vin))
print("cheap\t\t", cr, abs(cr - vin))


vincenty	 51.32327170545291
old		 51.24481546355443 0.0784562418984791
great_circle	 51.210650067719854 0.11262163773305645
cheap		 51.324820229117535 0.0015485236646242129
